In [ ]:
# 데이터 준비는 LSTM 모델과 동일
# 앞서 정의한 X_dict, y_dict를 사용


In [ ]:
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Flatten
from tensorflow.keras.models import Model

def create_transformer_model(input_shape):
    inputs = Input(shape=input_shape)
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x = Dropout(0.1)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Flatten()(x)
    outputs = Dense(1)(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [ ]:
# 모델 저장을 위한 딕셔너리
models = {}

for asset in asset_list:
    X = X_dict[asset]
    y = y_dict[asset]
    
    # 데이터 분할 (80% 학습, 20% 검증)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Transformer 입력 형태로 변환
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    # Transformer 모델 구축
    model = create_transformer_model(input_shape=(look_back, 1))
    
    # 조기 종료 콜백 설정
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    
    # 모델 학습
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=0)
    
    # 모델 저장
    models[asset] = model


In [ ]:
predicted_returns = {}

for asset in asset_list:
    X = X_dict[asset]
    y = y_dict[asset]
    
    # 전체 데이터를 사용하여 예측 수행
    X_all = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = models[asset]
    predictions = model.predict(X_all)
    
    # 역스케일링
    scaler = scalers[asset]
    predictions = scaler.inverse_transform(predictions)
    actual = scaler.inverse_transform(y.reshape(-1, 1))
    
    # 마지막 예측 값 저장
    predicted_returns[asset] = predictions[-1][0]
    
    # 모델 평가 (원한다면)
    # mse = mean_squared_error(actual, predictions)
    # print(f"{asset} - MSE: {mse}")


In [ ]:
# 위험 허용도 설정 (0에 가까울수록 보수적)
risk_tolerance = 0.2

# 변동성 계산
volatility = returns.std()

# 예상 수익률 데이터프레임 생성
predicted_returns_df = pd.DataFrame({
    'predicted_return': predicted_returns,
    'volatility': volatility
})

# 위험 대비 수익률 계산
predicted_returns_df['adjusted_score'] = predicted_returns_df['predicted_return'] / predicted_returns_df['volatility'] * risk_tolerance

# 가중치 계산 및 정규화
predicted_returns_df['weight'] = predicted_returns_df['adjusted_score'] / predicted_returns_df['adjusted_score'].sum()

# 추천 포트폴리오 출력
recommended_portfolio = predicted_returns_df[['predicted_return', 'volatility', 'weight']]
print("추천 포트폴리오 구성:")
print(recommended_portfolio.sort_values('weight', ascending=False))
